In [ ]:
import sys
import os
import time
sys.path.append(os.path.abspath('../..'))

import gpflow
import tensorflow as tf
from gpflow.optimizers import Scipy

from rcgp.morcgp import MOGPRegressor_NC, MORCGP_shared_noise
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
import pandas as pd

plt.rcParams.update({
    "text.usetex": True,         
    "font.family": "serif",       
    "text.latex.preamble": r"\usepackage{amsmath}",
    'font.size': 28,         
    'axes.labelsize': 28,    
    'axes.titlesize': 30,    
    'xtick.labelsize': 24,   
    'ytick.labelsize': 24,  
    'legend.fontsize': 24,
    'lines.linewidth': 5,    
    'lines.markersize': 6   
})

In [ ]:
def calculate_rmse(y_true, y_pred):
    errors = y_true - y_pred
    squared_errors = errors ** 2
    mse = np.mean(squared_errors)
    rmse = np.sqrt(mse)
    return rmse

def nlpd(Y_true, mu_pred, var_pred):
    epsilon = 1e-10
    var_pred = np.maximum(var_pred, epsilon)
    
    nlpd_values = 0.5 * np.log(2 * np.pi * var_pred) + ((Y_true - mu_pred) ** 2) / (2 * var_pred)
    
    return np.mean(nlpd_values)

In [ ]:
def uniform_outliers_c1(Y: np.ndarray, percent_outliers: float, start: float, end: float) -> np.ndarray:
    if not (0 <= percent_outliers <= 1):
        raise ValueError("percent_outliers must be between 0 and 1.")
    if start < 0 or end <= start:
        raise ValueError("Invalid range: ensure 0 <= start < end.")

    Y_outliers = Y.copy()
    N, D = Y.shape
    total_elements = N 
    num_outliers = int(np.round(percent_outliers * total_elements))

    row_indices = np.random.choice(N, num_outliers, replace=False)

    signs = np.random.choice([-1, 1], size=num_outliers)

    uniform_values = np.random.uniform(start, end, size=num_outliers) * signs

    Y_outliers[row_indices, 0] += uniform_values

    return Y_outliers

def asymmetric_outliers_c1(Y: np.ndarray, percent_outliers: float, start: float, end: float) -> np.ndarray:
    if not (0 <= percent_outliers <= 1):
        raise ValueError("percent_outliers must be between 0 and 1.")
    if start < 0 or end <= start:
        raise ValueError("Invalid range: ensure 0 <= start < end.")
    
    Y_outliers = Y.copy()
    N, D = Y.shape
    total_elements = N 
    num_outliers = int(np.round(percent_outliers * total_elements))

    row_indices = np.random.choice(N, num_outliers, replace=False)

    uniform_values = np.random.uniform(start, end, size=num_outliers)

    Y_outliers[row_indices, 0] += uniform_values

    return Y_outliers

def focused_outliers_c1(X, Y, percent_outliers, y_value, perturbation=0.1):
    def mad(X):
        medians = np.median(X, axis=0)
        deviations = np.abs(X - medians)
        return np.median(deviations, axis=0)

    X = X.copy()
    Y = Y.copy()

    n_samples, n_features = X.shape
    n_outliers = int(n_samples * percent_outliers)

    # Indices of outliers
    indices = np.random.choice(n_samples, size=n_outliers, replace=False)

    medians_2d = np.tile(np.median(X, axis=0), (n_outliers, 1))

    def mad(X, axis=0):
        """Compute Median Absolute Deviation (MAD)"""
        med = np.median(X, axis=axis)
        return np.median(np.abs(X - med), axis=axis)

    mads = mad(X)
    mads_2d = np.tile(mads, (n_outliers, 1))

    u = np.random.uniform(0, perturbation, size=medians_2d.shape)
    X_outliers = medians_2d + u * mads_2d

    Y_outliers = np.full(shape=n_outliers, fill_value=y_value)

    first_column = Y[:, 0]
    median_y0 = np.median(first_column)
    mad_y0 = np.median(np.abs(first_column - median_y0))
    Y_mad_outliers = np.full(shape=n_outliers, fill_value=mad_y0)

    # Draw u independently for each element
    u = np.random.uniform(0, perturbation, size=Y_outliers.shape)

    # Compute the perturbed Y_outliers
    Y_outliers_perturbed = Y_outliers + u * Y_mad_outliers

    X[indices, :] = X_outliers
    Y[indices, 0] = Y_outliers_perturbed

    return X, Y

In [ ]:
def make_X_multi(X, D=2):
    """
    X: shape (N, input_dim) - multi-dimensional input
    D: number of tasks
    """
    N, input_dim = X.shape
    X_multi = []
    
    for task in range(D):
        # Add task index as last column
        X_task = np.hstack([X, np.full((N, 1), task)])
        X_multi.append(X_task)
    
    return np.vstack(X_multi)  # Shape: (N*D, input_dim + 1)

def run_MOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled):

    start_total = time.time()

    X_multi_train = make_X_multi(X_train_scaled, D=2)
    X_multi_test = make_X_multi(X_test_scaled, D=2)
    Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
    Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

    input_dim = X_train_scaled.shape[1]  # number of features
    D = 2  # number of tasks

    base_kernel = gpflow.kernels.RBF(
        lengthscales=1.0,
        variance=0.1,
        active_dims=list(range(input_dim))
    )

    coregion_kernel = gpflow.kernels.Coregion(
        output_dim=D,
        rank=D,
        active_dims=[input_dim]
    )

    gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
    coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

    kernel = base_kernel * coregion_kernel

    model_gpr = gpflow.models.GPR(
        data=(X_multi_train, Y_multi_train),
        kernel=kernel,
        mean_function=None
    )

    gpflow.utilities.set_trainable(base_kernel.variance, False)

    opt = Scipy()

    def objective_closure_gpr():
        return -model_gpr.log_marginal_likelihood()
    try:
        opt.minimize(objective_closure_gpr, model_gpr.trainable_variables, options=dict(maxiter=1000))
    except Exception as e:
        print(f"Optimization failed: {e}")
        print("Try reducing maxiter or checking data shapes")

    mean_pred_mogp, var_pred_mogp = model_gpr.predict_y(X_multi_test)
    mu_mogp, std_mogp = mean_pred_mogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_mogp.numpy()).reshape(-1, D, order='F')

    end_total = time.time()
    time_mogp = end_total - start_total

    rmse_mogp = calculate_rmse(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'))
    nlpd_mogp = nlpd(Y_test_scaled, mu_mogp.reshape(-1, D, order='F'), std_mogp.reshape(-1, D, order='F')**2)
    
    return rmse_mogp, nlpd_mogp, time_mogp


def run_MORCGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers, k=1):
    start_total = time.time()

    mcd = MinCovDet(support_fraction=1-prop_outliers).fit(Y_train_scaled)
    robust_covariance = mcd.covariance_
    robust_init_A = np.linalg.cholesky(robust_covariance)

    morcgp = MORCGP_shared_noise(mean=0, length_scale=1, noise_var=0.1, A=robust_init_A)
    morcgp.fit(X_train_scaled, Y_train_scaled, epsilons=np.array([prop_outliers, 0]))
    morcgp.optimize_loo_cv(print_opt_param=False, print_iter_objective=False, k=k, init_cov=robust_covariance, fix_weights=True)

    mu_morcgp, var_morcgp = morcgp.predict(X_test_scaled)
    std_morcgp = np.sqrt(var_morcgp + morcgp.noise_var)
    end_total = time.time()

    time_morcgp = end_total - start_total
    rmse_morcgp = calculate_rmse(Y_test_scaled, mu_morcgp)
    nlpd_morcgp = nlpd(Y_test_scaled, mu_morcgp, std_morcgp**2)

    return rmse_morcgp, nlpd_morcgp, time_morcgp

def run_tMOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, df):

    X_multi_train = make_X_multi(X_train_scaled, D=2)
    X_multi_test = make_X_multi(X_test_scaled, D=2)
    Y_multi_train = Y_train_scaled.reshape(-1, 1, order='F')
    Y_multi_test = Y_test_scaled.reshape(-1, 1, order='F')

    input_dim = X_train_scaled.shape[1]  
    N = X_train_scaled.shape[0]
    D = 2

    start_total = time.time()

    base_kernel = gpflow.kernels.RBF(
        lengthscales=1.0, 
        variance=0.1, 
        active_dims=list(range(input_dim)) ,
    )

    coregion_kernel = gpflow.kernels.Coregion(
        output_dim=D, 
        rank=D, 
        active_dims=[input_dim]  
    )

    gpflow.utilities.set_trainable(base_kernel.variance, False)

    gpflow.utilities.set_trainable(coregion_kernel.kappa, False)
    coregion_kernel.kappa.assign(tf.ones_like(coregion_kernel.kappa) * 1e-6)

    kernel = base_kernel * coregion_kernel

    likelihood_vgp = gpflow.likelihoods.StudentT(df=df)
    model_vgp = gpflow.models.VGP(
        data=(X_multi_train, Y_multi_train),
        kernel=kernel,
        likelihood=likelihood_vgp
    )

    opt = Scipy()
    def objective_closure_vgp():
        return -model_vgp.maximum_log_likelihood_objective()

    try:
        opt.minimize(objective_closure_vgp, model_vgp.trainable_variables, options=dict(maxiter=1000))
    except Exception as e:
        print(f"Optimization failed: {e}")
        print("Try reducing maxiter or checking data shapes")

    mean_pred_tmogp, var_pred_tmogp = model_vgp.predict_y(X_multi_test)
    mu_tmogp, std_tmogp = mean_pred_tmogp.numpy().reshape(-1, D, order='F'), np.sqrt(var_pred_tmogp.numpy()).reshape(-1, D, order='F')
    end_total = time.time()

    time_tmogp = end_total - start_total
    rmse_tmogp = calculate_rmse(Y_test_scaled, mu_tmogp.reshape(-1, D, order='F'))
    nlpd_tmogp = nlpd(Y_test_scaled, mu_tmogp.reshape(-1, D, order='F'), std_tmogp.reshape(-1, D, order='F')**2)
    
    return rmse_tmogp, nlpd_tmogp, time_tmogp

def run_MOGP_numpy(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers):

    start_total = time.time()

    mcd = MinCovDet(support_fraction=1-prop_outliers).fit(Y_train_scaled)
    robust_covariance = mcd.covariance_
    robust_init_A = np.linalg.cholesky(robust_covariance)

    mogp = MOGPRegressor_NC(mean = 0, length_scale=1, noise = 0.1, A=robust_init_A)
    mogp.fit(X_train_scaled, Y_train_scaled)
    mogp.optimize_hyperparameters()

    mu_mogp, var_mogp = mogp.predict(X_test_scaled)
    std_mogp = np.sqrt(var_mogp + mogp.noise)
    end_total = time.time()

    time_mogp = end_total - start_total
    rmse_mogp = calculate_rmse(Y_test_scaled, mu_mogp)
    nlpd_mogp = nlpd(Y_test_scaled, mu_mogp, std_mogp**2)

    return rmse_mogp, nlpd_mogp, time_mogp


In [ ]:
# Import dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(url)
X = df.loc[:, 'X1':'X8'].to_numpy()
Y = df.loc[:, ['Y1', 'Y2']].to_numpy()

# No outliers

In [ ]:
rmses_mogp, rmses_morcgp, rmses_tmogp = [], [], []
nlpds_mogp, nlpds_morcgp, nlpds_tmogp = [], [], []
times_mogp, times_morcgp, times_tmogp = [], [], []

prop_outliers = 0
num_seeds = 20

run_mogp = False
run_morcgp = True
run_tmogp = False

for i in tqdm(range(num_seeds)):

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.25, random_state=i
    )

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    scaler_Y = StandardScaler()
    Y_train_scaled = scaler_Y.fit_transform(Y_train)
    Y_test_scaled = scaler_Y.transform(Y_test)

    if run_mogp:
        rmse_mogp, nlpd_mogp, time_mogp = run_MOGP_numpy(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers)
        print(f'No outliers: MOGP seed {i}: RMSE = {rmse_mogp}, NLPD = {nlpd_mogp}, Time = {time_mogp}')
        rmses_mogp.append(rmse_mogp)
        nlpds_mogp.append(nlpd_mogp)
        times_mogp.append(time_mogp)
    if run_morcgp:
        rmse_morcgp, nlpd_morcgp, time_morcgp = run_MORCGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers, k=2)
        print(f'No outliers: MORCGP seed {i}: RMSE = {rmse_morcgp}, NLPD = {nlpd_morcgp}, Time = {time_morcgp}')
        rmses_morcgp.append(rmse_morcgp)
        nlpds_morcgp.append(nlpd_morcgp)
        times_morcgp.append(time_morcgp)
    if run_tmogp:
        rmse_tmogp, nlpd_tmogp, time_tmogp = run_tMOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, df=10)
        print(f'No outliers: t-MOGP seed {i}: RMSE = {rmse_tmogp}, NLPD = {nlpd_tmogp}, Time = {time_tmogp}')
        rmses_tmogp.append(rmse_tmogp)
        nlpds_tmogp.append(nlpd_tmogp)
        times_tmogp.append(time_tmogp)

if run_mogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_mogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_mogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_mogp))
    print(f'RMSE MOGP: {np.mean(rmses_mogp):.4f} ± {np.std(rmses_mogp):.4f}')
    print(f'NLPD MOGP: {np.mean(nlpds_mogp):.4f} ± {np.std(nlpds_mogp):.4f}')
    print(f'Time MOGP: {np.mean(times_mogp):.4f} ± {np.std(times_mogp):.4f}')

if run_morcgp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_morcgp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_morcgp))
    print(f'RMSE MORCGP: {np.mean(rmses_morcgp):.4f} ± {np.std(rmses_morcgp):.4f}')
    print(f'NLPD MORCGP: {np.mean(nlpds_morcgp):.4f} ± {np.std(nlpds_morcgp):.4f}')
    print(f'Time MORCGP: {np.mean(times_morcgp):.4f} ± {np.std(times_morcgp):.4f}')

if run_tmogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_tmogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_tmogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_tmogp))
    print(f'RMSE t-MOGP: {np.mean(rmses_tmogp):.4f} ± {np.std(rmses_tmogp):.4f}')
    print(f'NLPD t-MOGP: {np.mean(nlpds_tmogp):.4f} ± {np.std(nlpds_tmogp):.4f}')
    print(f'Time t-MOGP: {np.mean(times_tmogp):.4f} ± {np.std(times_tmogp):.4f}')

  5%|▌         | 1/20 [01:17<24:38, 77.83s/it]

No outliers: MORCGP seed 0: RMSE = 0.12511164877270067, NLPD = -0.8502006690386331, Time = 77.82242465019226


 10%|█         | 2/20 [02:01<17:15, 57.52s/it]

No outliers: MORCGP seed 1: RMSE = 0.11565353556189453, NLPD = -0.7951018873299732, Time = 43.29043102264404


 15%|█▌        | 3/20 [02:20<11:20, 40.01s/it]

...

No outliers: MORCGP seed 17: RMSE = 0.10469056003379244, NLPD = -0.9846299217657357, Time = 36.16512179374695


 95%|█████████▌| 19/20 [14:03<00:44, 44.02s/it]

No outliers: MORCGP seed 18: RMSE = 0.1478571780966549, NLPD = -0.8211240999200956, Time = 18.880886554718018


100%|██████████| 20/20 [14:36<00:00, 43.81s/it]

No outliers: MORCGP seed 19: RMSE = 0.13276842555572813, NLPD = -0.7758588112911132, Time = 32.24533820152283
RMSE: 0.1251, 0.1157, 0.1126, 0.1032, 0.1036, 0.1111, 0.1263, 0.1250, 0.1087, 0.1186, 0.0972, 0.1374, 0.1294, 0.1363, 0.0903, 0.1241, 0.1008, 0.1047, 0.1479, 0.1328
NLPD: -0.8502, -0.7951, -0.8936, -0.9853, -0.9544, -0.8047, -0.6759, -0.8140, -0.9314, -0.9293, -0.9667, -0.6210, -0.7628, -0.7316, -1.0522, -0.8728, -0.9254, -0.9846, -0.8211, -0.7759
Time: 77.8224, 43.2904, 19.1723, 27.3977, 54.7897, 45.4805, 36.2380, 19.1061, 37.6373, 28.3980, 52.6968, 46.4609, 35.7727, 46.9938, 80.4785, 50.5067, 86.4893, 36.1651, 18.8809, 32.2453
RMSE MORCGP: 0.1175 ± 0.0149
NLPD MORCGP: -0.8574 ± 0.1093
Time MORCGP: 43.8011 ± 19.0859


# Uniform outliers

In [ ]:
rmses_mogp, rmses_morcgp, rmses_tmogp = [], [], []
nlpds_mogp, nlpds_morcgp, nlpds_tmogp = [], [], []
times_mogp, times_morcgp, times_tmogp = [], [], []

prop_outliers = 0.1
num_seeds = 20

run_mogp = True
run_morcgp = True
run_tmogp = True

for i in tqdm(range(num_seeds)):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.25, random_state=i
    )

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    scaler_Y = StandardScaler()
    Y_train_scaled = scaler_Y.fit_transform(Y_train)
    Y_test_scaled = scaler_Y.transform(Y_test)

    Y_train_scaled = uniform_outliers_c1(Y=Y_train_scaled, percent_outliers=prop_outliers, start=6, end=9)

    if run_mogp:
        rmse_mogp, nlpd_mogp, time_mogp = run_MOGP_numpy(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers)
        print(f'Uniform outliers: MOGP seed {i}: RMSE = {rmse_mogp}, NLPD = {nlpd_mogp}, Time = {time_mogp}')
        rmses_mogp.append(rmse_mogp)
        nlpds_mogp.append(nlpd_mogp)
        times_mogp.append(time_mogp)
    if run_morcgp:
        rmse_morcgp, nlpd_morcgp, time_morcgp = run_MORCGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers, k=2)
        print(f'Uniform outliers: MORCGP seed {i}: RMSE = {rmse_morcgp}, NLPD = {nlpd_morcgp}, Time = {time_morcgp}')
        rmses_morcgp.append(rmse_morcgp)
        nlpds_morcgp.append(nlpd_morcgp)
        times_morcgp.append(time_morcgp)
    if run_tmogp:
        rmse_tmogp, nlpd_tmogp, time_tmogp = run_tMOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, df=10)
        print(f'Uniform outliers: t-MOGP seed {i}: RMSE = {rmse_tmogp}, NLPD = {nlpd_tmogp}, Time = {time_tmogp}')
        rmses_tmogp.append(rmse_tmogp)
        nlpds_tmogp.append(nlpd_tmogp)
        times_tmogp.append(time_tmogp)

if run_mogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_mogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_mogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_mogp))
    print(f'RMSE MOGP: {np.mean(rmses_mogp):.4f} ± {np.std(rmses_mogp):.4f}')
    print(f'NLPD MOGP: {np.mean(nlpds_mogp):.4f} ± {np.std(nlpds_mogp):.4f}')
    print(f'Time MOGP: {np.mean(times_mogp):.4f} ± {np.std(times_mogp):.4f}')

if run_morcgp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print("Time:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print(f'RMSE MORCGP: {np.mean(rmses_morcgp):.4f} ± {np.std(rmses_morcgp):.4f}')
    print(f'NLPD MORCGP: {np.mean(nlpds_morcgp):.4f} ± {np.std(nlpds_morcgp):.4f}')
    print(f'Time MORCGP: {np.mean(times_morcgp):.4f} ± {np.std(times_morcgp):.4f}')

if run_tmogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_tmogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_tmogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_tmogp))
    print(f'RMSE t-MOGP: {np.mean(rmses_tmogp):.4f} ± {np.std(rmses_tmogp):.4f}')
    print(f'NLPD t-MOGP: {np.mean(nlpds_tmogp):.4f} ± {np.std(nlpds_tmogp):.4f}')
    print(f'Time t-MOGP: {np.mean(times_tmogp):.4f} ± {np.std(times_tmogp):.4f}')

  5%|▌         | 1/20 [00:32<10:23, 32.81s/it]

Uniform outliers: MOGP seed 0: RMSE = 1.5542110849362754, NLPD = 1.4419766746527696, Time = 32.803264141082764


 10%|█         | 2/20 [01:08<10:21, 34.55s/it]

Uniform outliers: MOGP seed 1: RMSE = 1.527532008372313, NLPD = 1.4763475681744307, Time = 35.76046180725098


 15%|█▌        | 3/20 [01:39<09:19, 32.91s/it]

...

Uniform outliers: MOGP seed 17: RMSE = 1.1040263137079893, NLPD = 1.3545628491892667, Time = 115.51184725761414


 95%|█████████▌| 19/20 [13:17<01:08, 68.54s/it]

Uniform outliers: MOGP seed 18: RMSE = 1.5997058021255672, NLPD = 1.4997181804320772, Time = 82.17452454566956


100%|██████████| 20/20 [13:49<00:00, 41.45s/it]

Uniform outliers: MOGP seed 19: RMSE = 1.3878196605750368, NLPD = 1.4421391249069189, Time = 31.922654390335083
RMSE: 1.5542, 1.5275, 1.0314, 1.1713, 1.5233, 1.7358, 1.3273, 1.5239, 1.3738, 1.6686, 1.5352, 1.2872, 1.2542, 1.0421, 1.5759, 1.2080, 1.3650, 1.1040, 1.5997, 1.3878
NLPD: 1.4420, 1.4763, 1.3810, 1.3693, 1.4467, 1.4876, 1.3664, 1.4370, 1.4413, 1.5296, 1.4275, 1.4062, 1.4436, 1.3850, 1.4500, 1.5210, 1.3504, 1.3546, 1.4997, 1.4421
Time: 32.8033, 35.7605, 30.9526, 34.4374, 34.4876, 34.3665, 34.5291, 34.3960, 34.3017, 35.8013, 34.3950, 34.1281, 30.7224, 30.6244, 34.0911, 47.3710, 46.1654, 115.5118, 82.1745, 31.9227
RMSE MOGP: 1.3898 ± 0.2030
NLPD MOGP: 1.4329 ± 0.0522
Time MOGP: 41.4471 ± 20.2822


# Asymmetric outliers

In [ ]:
rmses_mogp, rmses_morcgp, rmses_tmogp = [], [], []
nlpds_mogp, nlpds_morcgp, nlpds_tmogp = [], [], []
times_mogp, times_morcgp, times_tmogp = [], [], []

prop_outliers = 0.1
num_seeds = 20

run_mogp = True
run_morcgp = True
run_tmogp = True

for i in tqdm(range(num_seeds)):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.25, random_state=i
    )
    np.random.seed(i)

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    scaler_Y = StandardScaler()
    Y_train_scaled = scaler_Y.fit_transform(Y_train)
    Y_test_scaled = scaler_Y.transform(Y_test)

    Y_train_scaled = asymmetric_outliers_c1(Y=Y_train_scaled, percent_outliers=prop_outliers, start=6, end=9)

    if run_mogp:
        rmse_mogp, nlpd_mogp, time_mogp = run_MOGP_numpy(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers)
        print(f'Asymmetric outliers: MOGP seed {i}: RMSE = {rmse_mogp}, NLPD = {nlpd_mogp}, Time = {time_mogp}')
        rmses_mogp.append(rmse_mogp)
        nlpds_mogp.append(nlpd_mogp)
        times_mogp.append(time_mogp)
    if run_morcgp:
        rmse_morcgp, nlpd_morcgp, time_morcgp = run_MORCGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers, k=2)
        print(f'Asymmetric outliers: MORCGP seed {i}: RMSE = {rmse_morcgp}, NLPD = {nlpd_morcgp}, Time = {time_morcgp}')
        rmses_morcgp.append(rmse_morcgp)
        nlpds_morcgp.append(nlpd_morcgp)
        times_morcgp.append(time_morcgp)
    if run_tmogp:
        rmse_tmogp, nlpd_tmogp, time_tmogp = run_tMOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, df=10)
        print(f'Asymmetric outliers: t-MOGP seed {i}: RMSE = {rmse_tmogp}, NLPD = {nlpd_tmogp}, Time = {time_tmogp}')
        rmses_tmogp.append(rmse_tmogp)
        nlpds_tmogp.append(nlpd_tmogp)
        times_tmogp.append(time_tmogp)

if run_mogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_mogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_mogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_mogp))
    print(f'RMSE MOGP: {np.mean(rmses_mogp):.4f} ± {np.std(rmses_mogp):.4f}')
    print(f'NLPD MOGP: {np.mean(nlpds_mogp):.4f} ± {np.std(nlpds_mogp):.4f}')
    print(f'Time MOGP: {np.mean(times_mogp):.4f} ± {np.std(times_mogp):.4f}')

if run_morcgp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_morcgp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_morcgp))
    print(f'RMSE MORCGP: {np.mean(rmses_morcgp):.4f} ± {np.std(rmses_morcgp):.4f}')
    print(f'NLPD MORCGP: {np.mean(nlpds_morcgp):.4f} ± {np.std(nlpds_morcgp):.4f}')
    print(f'Time MORCGP: {np.mean(times_morcgp):.4f} ± {np.std(times_morcgp):.4f}')

if run_tmogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_tmogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_tmogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_tmogp))
    print(f'RMSE t-MOGP: {np.mean(rmses_tmogp):.4f} ± {np.std(rmses_tmogp):.4f}')
    print(f'NLPD t-MOGP: {np.mean(nlpds_tmogp):.4f} ± {np.std(nlpds_tmogp):.4f}')
    print(f'Time t-MOGP: {np.mean(times_tmogp):.4f} ± {np.std(times_tmogp):.4f}')

  5%|▌         | 1/20 [00:41<13:02, 41.19s/it]

Asymmetric outliers: MOGP seed 0: RMSE = 1.2263304750818784, NLPD = 1.4241092177249548, Time = 41.186551094055176


 10%|█         | 2/20 [01:33<14:21, 47.87s/it]

Asymmetric outliers: MOGP seed 1: RMSE = 1.3671313238587208, NLPD = 1.5070556858763287, Time = 52.54415678977966


 15%|█▌        | 3/20 [02:08<11:49, 41.74s/it]

...

Asymmetric outliers: MOGP seed 17: RMSE = 1.6825416372176636, NLPD = 1.4949157206228854, Time = 27.756648778915405


 95%|█████████▌| 19/20 [13:03<00:33, 33.53s/it]

Asymmetric outliers: MOGP seed 18: RMSE = 1.6052106791445426, NLPD = 1.4822330993416888, Time = 35.07787466049194


100%|██████████| 20/20 [13:36<00:00, 40.84s/it]

Asymmetric outliers: MOGP seed 19: RMSE = 1.6201740302052063, NLPD = 1.4872963932345176, Time = 33.575167417526245
RMSE: 1.2263, 1.3671, 1.4691, 1.3759, 1.4135, 1.5096, 1.5883, 1.4185, 1.7209, 2.4971, 1.3530, 1.3290, 1.5078, 1.6373, 1.3879, 1.3018, 1.5055, 1.6825, 1.6052, 1.6202
NLPD: 1.4241, 1.5071, 1.4132, 1.4768, 1.3942, 1.4250, 1.4603, 1.4155, 1.5477, 1.8835, 1.4045, 1.4292, 1.4251, 1.4470, 1.4209, 1.4288, 1.4484, 1.4949, 1.4822, 1.4873
Time: 41.1866, 52.5442, 34.4199, 31.4717, 34.1180, 34.3539, 65.4119, 117.9874, 35.1124, 42.0574, 34.1868, 27.7749, 34.4796, 33.7133, 34.7482, 32.8326, 33.8056, 27.7566, 35.0779, 33.5752
RMSE MOGP: 1.5258 ± 0.2589
NLPD MOGP: 1.4708 ± 0.1022
Time MOGP: 40.8307 ± 19.5668


# Focused outliers

In [ ]:
rmses_mogp, rmses_morcgp, rmses_tmogp = [], [], []
nlpds_mogp, nlpds_morcgp, nlpds_tmogp = [], [], []
times_mogp, times_morcgp, times_tmogp = [], [], []

prop_outliers = 0.1
num_seeds = 20

run_mogp = True
run_morcgp = True
run_tmogp = True

for i in tqdm(range(num_seeds)):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.25, random_state=i
    )

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    scaler_Y = StandardScaler()
    Y_train_scaled = scaler_Y.fit_transform(Y_train)
    Y_test_scaled = scaler_Y.transform(Y_test)

    X_train_scaled, Y_train_scaled = focused_outliers_c1(X=X_train_scaled, Y=Y_train_scaled, percent_outliers=prop_outliers, y_value=6, perturbation=0.1)

    if run_mogp:
        rmse_mogp, nlpd_mogp, time_mogp = run_MOGP_numpy(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers)
        print(f'Focused outliers: MOGP seed {i}: RMSE = {rmse_mogp}, NLPD = {nlpd_mogp}, Time = {time_mogp}')
        rmses_mogp.append(rmse_mogp)
        nlpds_mogp.append(nlpd_mogp)
        times_mogp.append(time_mogp)
    if run_morcgp:
        rmse_morcgp, nlpd_morcgp, time_morcgp = run_MORCGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, prop_outliers=prop_outliers, k=2)
        print(f'Focused outliers: MORCGP seed {i}: RMSE = {rmse_morcgp}, NLPD = {nlpd_morcgp}, Time = {time_morcgp}')
        rmses_morcgp.append(rmse_morcgp)
        nlpds_morcgp.append(nlpd_morcgp)
        times_morcgp.append(time_morcgp)
    if run_tmogp:
        rmse_tmogp, nlpd_tmogp, time_tmogp = run_tMOGP(X_train_scaled, Y_train_scaled, X_test_scaled, Y_test_scaled, df=9)
        print(f'Focused outliers: t-MOGP seed {i}: RMSE = {rmse_tmogp}, NLPD = {nlpd_tmogp}, Time = {time_tmogp}')
        rmses_tmogp.append(rmse_tmogp)
        nlpds_tmogp.append(nlpd_tmogp)
        times_tmogp.append(time_tmogp)

if run_mogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_mogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_mogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_mogp))
    print(f'RMSE MOGP: {np.mean(rmses_mogp):.4f} ± {np.std(rmses_mogp):.4f}')
    print(f'NLPD MOGP: {np.mean(nlpds_mogp):.4f} ± {np.std(nlpds_mogp):.4f}')
    print(f'Time MOGP: {np.mean(times_mogp):.4f} ± {np.std(times_mogp):.4f}')

if run_morcgp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_morcgp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_morcgp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_morcgp))
    print(f'RMSE MORCGP: {np.mean(rmses_morcgp):.4f} ± {np.std(rmses_morcgp):.4f}')
    print(f'NLPD MORCGP: {np.mean(nlpds_morcgp):.4f} ± {np.std(nlpds_morcgp):.4f}')
    print(f'Time MORCGP: {np.mean(times_morcgp):.4f} ± {np.std(times_morcgp):.4f}')

if run_tmogp:
    print("RMSE:", ", ".join(f"{x:.4f}" for x in rmses_tmogp))
    print("NLPD:", ", ".join(f"{x:.4f}" for x in nlpds_tmogp))
    print("Time:", ", ".join(f"{x:.4f}" for x in times_tmogp))
    print(f'RMSE t-MOGP: {np.mean(rmses_tmogp):.4f} ± {np.std(rmses_tmogp):.4f}')
    print(f'NLPD t-MOGP: {np.mean(nlpds_tmogp):.4f} ± {np.std(nlpds_tmogp):.4f}')
    print(f'Time t-MOGP: {np.mean(times_tmogp):.4f} ± {np.std(times_tmogp):.4f}')

  5%|▌         | 1/20 [00:42<13:25, 42.40s/it]

Focused outliers: MOGP seed 0: RMSE = 0.19255731324752393, NLPD = 0.021438826667354693, Time = 42.40018033981323


 10%|█         | 2/20 [01:13<10:44, 35.78s/it]

Focused outliers: MOGP seed 1: RMSE = 0.21304126260538378, NLPD = -0.004859689026343435, Time = 31.145109176635742


 15%|█▌        | 3/20 [01:44<09:29, 33.51s/it]

...

Focused outliers: MOGP seed 17: RMSE = 0.16959236487502186, NLPD = -0.061634861591770986, Time = 73.38109636306763


 95%|█████████▌| 19/20 [15:11<01:00, 60.52s/it]

Focused outliers: MOGP seed 18: RMSE = 0.18817501830332728, NLPD = 0.04136644524623611, Time = 75.7829122543335


100%|██████████| 20/20 [15:54<00:00, 47.70s/it]

Focused outliers: MOGP seed 19: RMSE = 0.2054788781425501, NLPD = 0.024888619659890945, Time = 42.63155961036682
RMSE: 0.1926, 0.2130, 0.2078, 0.2104, 0.1721, 0.1811, 0.1789, 0.1870, 0.1914, 0.2072, 0.1642, 0.1660, 0.2101, 0.1788, 0.1638, 0.2000, 0.1835, 0.1696, 0.1882, 0.2055
NLPD: 0.0214, -0.0049, 0.0146, 0.0412, 0.0461, 0.0233, -0.0536, 0.0420, 0.0400, 0.0733, 0.0401, -0.1380, 0.0665, -0.0852, 0.0147, -0.0477, 0.0307, -0.0616, 0.0414, 0.0249
Time: 42.4002, 31.1451, 30.7974, 78.5144, 33.5454, 21.8565, 57.5175, 61.4166, 21.1836, 28.0592, 21.3794, 91.0886, 75.2422, 63.2028, 21.8592, 30.9643, 51.9118, 73.3811, 75.7829, 42.6316
RMSE MOGP: 0.1886 ± 0.0163
NLPD MOGP: 0.0065 ± 0.0537
Time MOGP: 47.6940 ± 22.1277
